In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

from jointSBM import jointSBM

import warnings
warnings.filterwarnings('ignore')

In [2]:
dir_path = "data/N_100_fixed_200_alpha_20/"
edgelistsFiles = [os.path.join(dir_path,f) for f in os.listdir(dir_path) if f.find("edge")>0]
membFiles = [os.path.join(dir_path,f) for f in os.listdir(dir_path) if f.find("memb")>0]

In [3]:
graphs = {}
for f in edgelistsFiles:
    graphName = os.path.split(f)[1].split(".")[0]
    graphs[graphName] = pd.read_csv(f).values-1.

In [4]:
groundTruth = {}
for f in membFiles:
    graphName = os.path.split(f)[1].split(".")[0]
    groundTruth[graphName] = pd.read_csv(f).values[:,1]

In [5]:
a = jointSBM(graphs, 6, groundTruth = groundTruth, n = 200, symmetric = False)
a.prepare_data()

a.fit(printLoss = True,parallel = True)

  0%|          | 0/100 [00:00<?, ?it/s]

Converting edgelists to sparse adjacency matrices...


100%|██████████| 100/100 [00:04<00:00, 21.37it/s]


Iter: 1 | Loss: 7356113.362123064


Iter: 1 | Loss: 7356113.362123064


...

Iter: 1 | Loss: 7356113.362123064


Iter: 1 | Loss: 7356113.362123064


Iter: 1 | Loss: 7356113.362123064


Iter: 1 | Loss: 7356113.362123064
Iter: 2 | Loss: 7321926.128141775
Iter: 3 | Loss: 7360782.534459807
Iter: 4 | Loss: 7313737.009955663
Iter: 5 | Loss: 7318835.989789035
Iter: 6 | Loss: 7353474.776231994
Iter: 7 | Loss: 7317937.122399776
Iter: 8 | Loss: 7306671.8753463
Iter: 9 | Loss: 3211034.0240978943
Iter: 10 | Loss: 1979180.1251681182
Iter: 11 | Loss: 1734487.6995202536
Iter: 12 | Loss: 1718719.3399872868
Iter: 13 | Loss: 1706734.4536393676
Iter: 14 | Loss: 1689314.0060873632
Iter: 15 | Loss: 1816853.9557632147
Iter: 16 | Loss: 1811710.132150622
Iter: 17 | Loss: 2108654.547868938
Iter: 18 | Loss: 2230439.5531129926
Iter: 19 | Loss: 2251065.853556875
Iter: 20 | Loss: 2259738.1470297608
Iter: 21 | Loss: 2250533.9921917473
Iter: 22 | Loss: 2261051.1373062795
Iter: 23 | Loss: 2315660.528072847
Iter: 24 | Loss: 2348298.4550890555
Iter: 25 | Loss: 2345242.033462619
Iter: 26 | Loss: 2350958.4378304137
Iter: 27 | Loss: 2343937.4416860174
Iter: 28 | Loss: 2345014.650314569
Iter: 29 | Loss: 

({'Graph16': {1: 3,
   2: 3,
   3: 5,
   4: 3,
   5: 3,
   6: 3,
   7: 3,
   8: 3,
   9: 3,
   10: 3,
   11: 3,
   12: 3,
   13: 3,
   14: 3,
   15: 3,
   16: 3,
   17: 3,
   18: 3,
   19: 3,
   20: 3,
   21: 3,
   22: 3,
   23: 3,
   24: 3,
   25: 3,
   26: 3,
   27: 1,
   28: 1,
   29: 3,
   30: 3,
   31: 3,
   32: 3,
   33: 3,
   34: 3,
   35: 3,
   36: 3,
   37: 3,
   38: 3,
   39: 3,
   40: 1,
   41: 3,
   42: 3,
   43: 3,
   44: 3,
   45: 3,
   46: 1,
   47: 3,
   48: 3,
   49: 3,
   50: 3,
   51: 3,
   52: 3,
   53: 3,
   54: 3,
   55: 3,
   56: 3,
   57: 3,
   58: 3,
   59: 3,
   60: 1,
   61: 3,
   62: 3,
   63: 3,
   64: 3,
   65: 1,
   66: 3,
   67: 3,
   68: 3,
   69: 3,
   70: 3,
   71: 3,
   72: 3,
   73: 3,
   74: 3,
   75: 3,
   76: 3,
   77: 3,
   78: 3,
   79: 3,
   80: 3,
   81: 3,
   82: 3,
   83: 1,
   84: 3,
   85: 3,
   86: 3,
   87: 3,
   88: 3,
   89: 3,
   90: 3,
   91: 3,
   92: 5,
   93: 3,
   94: 3,
   95: 3,
   96: 3,
   97: 1,
   98: 1,
   99: 3,
   100: 

In [6]:
print(a.measures[a.iter])

{'NMI': {'nmi': 0.5421419069061393, 'overall_nmi': 0.5905683418237193}, 'ARI': {'ari': 0.6635532781212256, 'overall_ari': 0.5262029338166636}, 'MCR': {'mcr': 0.9177500000000001, 'overall_mcr': 0.79845}, 'Time': 0.3585038185119629}
